In [1]:
from pyquery import PyQuery as pq
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
from pprint import PrettyPrinter as pp

In [2]:
with open('../candidate_pages.json') as data_file:    
    candidate_json = json.load(data_file)

In [3]:
ROOT_LINK = "http://myneta.info"

In [4]:
for key,val in candidate_json.iteritems():
    if isinstance( val, int ):
        print key
    elif len(val) < 5:
        print key

In [18]:
def get_otherelec_link(page_text, url):
    for a in pq(page_text)("a"):
        if a.text == "Click here for more details":
            other_elec_link = pq(a).attr.href
            return ROOT_LINK+other_elec_link
    return False

test_index = "http://myneta.info/ls2009/candidate.php?candidate_id=1245"
page_data = candidate_json[test_index]
other_link = get_otherelec_link(page_data,test_index)


In [29]:
def get_otherelec_data(otherelec_link):
    
    otherelec_dict = {'common_link': otherelec_link}
    
    html = requests.get(otherelec_link)
    doc = pq(html.content)
    
    columns = []
    all_dicts = []
    add = 0
    trs = doc('tr')
    for tr in trs:
        elec_dict = otherelec_dict.copy()
        for th in pq(tr)('th'):
            columns.append(pq(th).text().replace(" ","_"))
            add = 0
        for i,td in enumerate(pq(tr)('td')):
            a = pq(td)('a')
            if a:
                elec_dict['elec_link'] = ROOT_LINK+a.attr.href
                elec_dict[columns[i]] = a.text()
            else:
                try:
                    if pq(td)('span') and i < 6:
                        elec_dict[columns[i]] = pq(td)('span').text()
                    else:
                        elec_dict[columns[i]] = str(pq(td).contents()[0]).encode('utf-8').strip().replace(',','')
                except:
                    print ""
                    print "Skipping col %s for %s" % (columns[i], elec_dict['common_link'])
            add = 1
            
        if add == 1:
            all_dicts.append(elec_dict)
    
    return all_dicts

get_otherelec_data(other_link)

[{'Age': '71',
  'Constituency': 'Hazaribagh',
  'Criminal_Cases': 'Yes',
  'Education_Level': 'Post Graduate',
  'Name': 'YASHWANT SINHA in Lok Sabha 2009',
  'Number_of_Cases': '1',
  'PAN_Given_(Y_or_N)': 'Y',
  'Party_Code': 'BJP',
  'Total_Assets': '36576217',
  'Total_Liabilities': '0',
  'common_link': 'http://myneta.info/compare_profile.php?group_id=6ZTQyN2EV2kbL4FLA8dP',
  'elec_link': 'http://myneta.info/ls2009/candidate.php?candidate_id=1245'},
 {'Age': '66',
  'Constituency': 'Hazaribagh',
  'Criminal_Cases': 'No',
  'Education_Level': 'Post Graduate',
  'Name': 'YASHWANT SINHA in Lok Sabha 2004',
  'Number_of_Cases': '0',
  'PAN_Given_(Y_or_N)': 'Y',
  'Party_Code': 'BJP',
  'Total_Assets': '15637769',
  'Total_Liabilities': '0',
  'common_link': 'http://myneta.info/compare_profile.php?group_id=6ZTQyN2EV2kbL4FLA8dP',
  'elec_link': 'http://myneta.info/loksabha2004/candidate.php?candidate_id=1529'}]

In [30]:
def get_all_elec():
    all_elec_data = []   
    counter = 0.0
    for key, val in candidate_json.iteritems():
        thelink = get_otherelec_link(val,key)
        counter += 1
        if counter%100 == 0.0:
            print ".",
        if thelink:
            all_elec_data = all_elec_data + get_otherelec_data(thelink)
    
    df = pd.DataFrame(all_elec_data)
    return df.drop_duplicates()

all_elecs_df = get_all_elec()
all_elecs_df.head(3)['common_link']

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Skipping col Constituency for http://myneta.info/compare_profile.php?group_id=JqXzgmSjQcXmaqJ9SSz8
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

0    http://myneta.info/compare_profile.php?group_i...
1    http://myneta.info/compare_profile.php?group_i...
2    http://myneta.info/compare_profile.php?group_i...
Name: common_link, dtype: object

In [31]:
all_elecs_df.head(3)

,Age,Constituency,Criminal_Cases,Education_Level,Name,Number_of_Cases,PAN_Given_(Y_or_N),Party_Code,Total_Assets,Total_Liabilities,common_link,elec_link
0,71,Hazaribagh,Yes,Post Graduate,YASHWANT SINHA in Lok Sabha 2009,1,Y,BJP,36576217,0,http://myneta.info/compare_profile.php?group_i...,http://myneta.info/ls2009/candidate.php?candid...
1,66,Hazaribagh,No,Post Graduate,YASHWANT SINHA in Lok Sabha 2004,0,Y,BJP,15637769,0,http://myneta.info/compare_profile.php?group_i...,http://myneta.info/loksabha2004/candidate.php?...
2,78,SHIMLA RURAL,Yes,Post Graduate,Virbhadra Singh in Himachal Pradesh 2012,1,Y,INC,336564971,2402720,http://myneta.info/compare_profile.php?group_i...,http://myneta.info/hp2012/candidate.php?candid...


In [32]:
all_elecs_df.to_csv('../multiple_elections.csv',encoding='utf-8')